# Задание 1


In [30]:
import pandas as pd
from scipy import stats as st
import matplotlib.pyplot as plt  #Пакет для построения графиков и прочего
import seaborn as sns

df = pd.read_csv('vgsales.csv')
df = df[['Platform', 'Genre', 'Critic_Score']]
df

,Platform,Genre,Critic_Score
0,Wii,Sports,76.0
1,NES,Platform,NaN
2,Wii,Racing,82.0
3,Wii,Sports,80.0
4,GB,Role-Playing,NaN
...,...,...,...
16714,PS3,Action,NaN
16715,X360,Sports,NaN
16716,PSV,Adventure,NaN
16717,GBA,Platform,NaN


1 вопрос: Как критики относятся к спортивным играм?

Н0: критики оценивают спортивные игры на уровень <=70, они не в восторге от этих игр.
Н1: критики любят спортивные игры на уровень > 70
Уровень значимости возьмем 0.05

In [21]:
result = st.ttest_1samp(df[df['Genre'] == 'Sports']['Critic_Score'].dropna(), 75, alternative='greater')
print(result)

Ttest_1sampResult(statistic=-7.5485926388546725, pvalue=0.9999999999999565)


вывод: pvalue > 0.05, считаем, что критики не в восторге от спортивных игр (в среднем оценки менее 75), аргументов отвергнуть Н0 пока не достаточно.

2 вопрос: Критикам нравятся больше игры на PC или на PS4?

Н0: критики любят игры а PC также, как и игры на PS4.
Н1: критики отдают кому-то предпочтение.
Уровень значимости возьмем 0.05

In [29]:
result = st.ttest_ind(df[df['Platform'] == 'PC']['Critic_Score'].dropna(), 
                      df[df['Platform'] == 'PS4']['Critic_Score'].dropna(), equal_var=False)

print(result)
df[df['Platform'] == 'PC'][['Critic_Score']].dropna().describe(), df[df['Platform'] == 'PS4'][['Critic_Score']].dropna().describe()

Ttest_indResult(statistic=4.3087588262138725, pvalue=2.067249157283479e-05)


(       Critic_Score
 count    715.000000
 mean      75.928671
 std       11.170775
 min       33.000000
 25%       70.000000
 50%       78.000000
 75%       84.000000
 max       96.000000,
        Critic_Score
 count    252.000000
 mean      72.091270
 std       12.485992
 min       19.000000
 25%       66.000000
 50%       73.000000
 75%       80.000000
 max       97.000000)

вывод: pvalue < 0.05, следовательно, можем считать, что критики отдают кому-то предпочтение, statistic > 0, следовательно, предпочтение отдается играм на PC

3 вопрос: Критикам больше нравятся стрелялки или стратегии?

Н0: критики любят стрелялки и стратегии одинаково.
Н1: критики отдают кому-то предпочтение.
Уровень значимости возьмем 0.05

In [32]:
result = st.ttest_ind(df[df['Genre'] == 'Shooter']['Critic_Score'].dropna(), 
                      df[df['Genre'] == 'Strategy']['Critic_Score'].dropna(), equal_var=False)

print(result)
df[df['Genre'] == 'Shooter'][['Critic_Score']].dropna().describe(), df[df['Genre'] == 'Strategy'][['Critic_Score']].dropna().describe()

Ttest_indResult(statistic=-2.2972408230640315, pvalue=0.021938989522304823)


(       Critic_Score
 count    944.000000
 mean      70.181144
 std       14.552571
 min       22.000000
 25%       61.000000
 50%       73.000000
 75%       81.000000
 max       97.000000,
        Critic_Score
 count    302.000000
 mean      72.086093
 std       11.828475
 min       35.000000
 25%       65.000000
 50%       73.000000
 75%       80.000000
 max       94.000000)

вывод: pvalue < 0.05, следовательно, можем считать, что критики отдают кому-то предпочтение, statistic < 0, следовательно, предпочтение отдается стратегиям. Если уровень значимости приняли бы за 0.01, то Н0 не отвергли, можно было бы считать, что стратегии и стрелялки критики любят одинаково.

# Задание 2

In [107]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package stopwords to C:\Users\Царь и
[nltk_data]     Бог\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Царь и
[nltk_data]     Бог\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [108]:
df = pd.read_csv('spam.csv')
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [109]:
def prepare_text(message):
    message = message.lower() 
    message = re.sub('[\W_]+',' ', message)
    message_split = message.split()
    stopwords_set = set(stopwords.words('english'))
    without_sw = [word for word in message_split if word not in stopwords_set]
    
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_text = [wordnet_lemmatizer.lemmatize(word) for word in without_sw]
    return ' '.join(lemmatized_text)

def bin_result(category):
    if category == 'ham':
        return 1
    else:
        return 0

df['prep_mess'] = df['Message'].apply(prepare_text)
df['bin'] = df['Category'].apply(bin_result)
print(df['bin'].value_counts())
df.head(20)

1    4825
0     747
Name: bin, dtype: int64


,Category,Message,prep_mess,bin
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,1
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,1
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,0
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,1
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think go usf life around though,1
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darling 3 week word back like fun ...,0
6,ham,Even my brother is not like to speak with me. ...,even brother like speak treat like aid patent,1
7,ham,As per your request 'Melle Melle (Oru Minnamin...,per request melle melle oru minnaminunginte nu...,1
8,spam,WINNER!! As a valued network customer you have...,winner valued network customer selected receiv...,0
9,spam,Had your mobile 11 months or more? U R entitle...,mobile 11 month u r entitled update latest col...,0


In [110]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['prep_mess'])
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names) 
tfidf_matrix

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
x_train, x_test, y_train, y_test = train_test_split(tfidf_matrix, df['bin'], test_size=0.30, random_state=42)
x_train.shape
y_train.shape
model = LogisticRegression(random_state=42)
model.fit(x_train, y_train) 
y_pred = model.predict(x_test)

model.score(x_test, y_test)

#confusion_matrix(y_true, y_pred)

0.958732057416268

получили очень хорошую точность!!! но классы сообщений (спам/не спам) не сбалансированы (747/4825)

In [112]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[ 158   66]
 [   3 1445]]


верных прогнозов 158+1445, ошибочных 3+66; видно, что ошибок FP больше, чем ошибок FN, т.к. не спама было больше в данных, его модель определяет лучше.